数据库的改动往往会引起数据的丢失等问题,而在开发阶段有不可避免的要对数据库进行修改这时候数据迁移技术就有用了

 [Alembic](https://alembic.readthedocs.org/en/latest/index.html)是常用的数据迁移框架,而[`Flask-Migrate`](http://flask-migrate.readthedocs.org/en/latest/)是一个对它的轻量级包装,我们可以直接用它在`Flask-Script`中实现数据库迁移.

In [1]:
!pip3 install flask-migrate

    100% |████████████████████████████████| 954kB 444kB/s 
    100% |████████████████████████████████| 565kB 278kB/s 
  Running setup.py install for Mako
  Running setup.py install for python-editor
  Running setup.py install for alembic
  Running setup.py install for flask-migrate
You are using pip version 7.1.2, however version 8.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#  创建迁移仓库

我们要做的只是把Migrate初始化,之后添加到manager的命令中

In [2]:
%%writefile ../codes/msgboard/manager.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A startup manager of the application.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

应用的启动文件

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""

__author__ = "Huang Sizhe"
__date__ = "22/01/2016"

import os
import sys
from flask.ext.script import Manager,Shell
from flask.ext.migrate import Migrate, MigrateCommand

root = os.path.dirname(__file__)
#把新加的表名放进去便于操作
from app import app,db,Message,User,Role
from config import AdminAccount

manager = Manager(app)
migrate = Migrate(app, db)# 初始化Migrate
def init_db(db):
    db.create_all()
    admin_role = Role(name='Admin')
    mod_role = Role(name='Moderator')
    user_role = Role(name='User')
    db.session.add_all([admin_role, mod_role, user_role])
    for i in AdminAccount:
        adminaccount = User(name=i["name"] ,email=i["email"],role =admin_role ,password = i["password"])
        db.session.add(adminaccount)
    db.session.commit()


def make_shell_context():
    return dict(app=app, 
                db=db,
                Message=Message,
                User=User,
                Role=Role,
                init_db=init_db)

manager.add_command("shell", Shell(make_context=make_shell_context))
manager.add_command('db', MigrateCommand)

if __name__ == '__main__':
    manager.run()

Overwriting ../codes/msgboard/manager.py


为了导出数据库迁移命令,Flask-Migrate 提供了一个 MigrateCommand 类,可附加到 Flask- Script 的 manager 对象上。在这个例子中,MigrateCommand 类使用 db 命令附加。

在维护数据库迁移之前,要使用 init 子命令创建迁移仓库:

    python manager.py db init

这个命令会创建 migrations 文件夹,所有迁移脚本都存放其中。数据库迁移仓库中的文件要和程序的其他文件一起纳入版本控制。

# 创建迁移脚本 

在 Alembic 中,数据库迁移用迁移脚本表示。脚本中有两个函数,分别是 upgrade() 和 downgrade()。upgrade() 函数把迁移中的改动应用到数据库中,downgrade() 函数则将改动 删除。Alembic 具有添加和删除改动的能力,因此数据库可重设到修改历史的任意一点。

我们可以使用 revision 命令手动创建 Alembic 迁移,也可使用 migrate 命令自动创建。 手动创建的迁移只是一个骨架,upgrade() 和 downgrade() 函数都是空的,开发者要使用Alembic 提供的 Operations 对象指令实现具体操作。自动创建的迁移会根据模型定义和数 据库当前状态之间的差异生成 upgrade() 和 downgrade() 函数的内容。

自动创建的迁移不一定总是正确的,有可能会漏掉一些细节。自动生成迁移 脚本后一定要进行检查。

migrate 子命令用来自动创建迁移脚本:

    python manager.py db migrate -m "initial migration"

# 更新数据库

    python manager.py db upgrade

# 总结

这部分我们介绍了下数据库迁移

用到的模块:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展
flask-wtf|构建表单
Flask-Bcrypt|密码加密
flask-SQLAlchemy|关系数据库ORM
Flask-Login|登录管理
itsdangerous(flask自带)|生成验证token
flask-mail(可选)|发送邮件
Flask-Migrate|数据库迁移

小型app的搭建攻略就到此为止了,这部分主要是介绍数据库操作和单app多文件的项目架构.学了这个,一般的小型app已经足够应付了,但也可以看到随着功能的增加,代码的增多,项目的复杂度变高了,可读性也变差了,之后的部分我们将学习如何用蓝图构建大中型项目,但在之前,让我们先扩展一下,用flask写个api服务(请看EX01)